In [ ]:
#Python standard library imports
import datetime as dt
import pathlib
import re
#Scientific Python ecosystem imports
import pandas as pd
#Text mining packages
import nltk as nltk
#from whoosh  import stem
#MM Import
import datetime as dt
import pathlib
import logging

from docx2python import docx2python

from polmap.polmap import prepare_keywords, doc2text # replaced the keyword processing block

In [ ]:
#Set Options for pandas visualization
pd.set_option('max_columns', None)
pd.set_option('max_colwidth', None)
pd.set_option('max_rows', None)
pd.set_option('max_colwidth', None)
pd.set_option('max_seq_item', None)
pd.set_option('precision', 3)

In [ ]:
#Create output folders
date = dt.datetime.now().date().isoformat()
time = dt.datetime.now().time().isoformat(timespec='seconds').replace(':', '')
current_date = '_'+date+'_T'+time

project_title = 'Export_Projects_Overview_08012021_162543'+str(current_date)

out_dir = pathlib.Path.cwd() / 'output' / project_title 
log_dir = out_dir / 'logs'
results_dir = out_dir / 'results'
docs2txt_dir = out_dir / 'docs2txt'

dir_dict = { directory: directory.mkdir(mode=0o777, parents=True, exist_ok=True) for directory in [out_dir, log_dir, results_dir] }

In [ ]:
#Create logifle for run
log_file = log_dir / 'mapping_{}.log'.format(project_title)
log_file.touch(mode=0o666)
logging.basicConfig(filename=log_file, filemode='a', level=logging.WARNING)

In [ ]:
#Define input dir
input_dir = pathlib.Path.cwd() / 'pdf_re' / 'Export_Projects_Overview_08012021_162543' #MM let user provide an input dir
input_folder_name = input_dir.name

allowed_filetypes=['.pdf','.html','.mhtml','.doc','.docx']

files = sorted(input_dir.glob('**/*.*'))
files = [ file for file in files if file.suffix in allowed_filetypes]

In [ ]:
class PdfConverter:
   def __init__(self, file_path):
       self.file_path = file_path
# convert pdf file to a string which has space among words
   def convert_pdf_to_txt(self):
       rsrcmgr = PDFResourceManager()
       retstr = StringIO()
       laparams = LAParams()
       logging.propagate = False
       logging.getLogger().setLevel(logging.ERROR)
       device = TextConverter(rsrcmgr, retstr, laparams=laparams)
       fp = open(self.file_path, 'rb')
       interpreter = PDFPageInterpreter(rsrcmgr, device)
       password = ""
       maxpages = 0
       caching = True
       pagenos = set()
       for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password, caching=caching, check_extractable=True):
           interpreter.process_page(page)
       fp.close()
       device.close()
       str = retstr.getvalue()
       retstr.close()
       return str

In [ ]:
PDFtext = []
counter = 0
for doc_item in files:
    print(doc_item)
    counter += 1
    try:
        policy_text=[]
        doc_text = doc2text(doc_item)
        while '\n\n\n\n' in doc_text : doc_text = doc_text.replace('\n\n\n\n', '\n\n\n')
        policy_text.append(doc_text)
        doc_item_name = '/'.join(doc_item.parts[doc_item.parts.index(input_dir.name)+1:]) #the path string of each file, including all parent directories except that are subdirectories of the input directory. It basically capture the directory tree 
        doc_filename = docs2txt_dir.joinpath(doc_item.parts[-2])
        doc_filename.mkdir(mode=0o777, parents=True, exist_ok=True)
        doc_filename = doc_filename.joinpath(doc_item.stem+'.txt')
        with open(doc_filename, 'w') as doctext:
           doctext.write(doc_text)
        PDFtext.append([doc_item_name,' ; '.join(policy_text)])
    except Exception as excptn: #MM I'd log errors as described in https://realpython.com/python-logging/, we need to test this.
        logging.exception('{doc_file} raised exception {exception} \n\n'.format(doc_file=doc_item.name, exception=excptn))